# VoxelOps Neuroimaging Pipeline Tutorial

This notebook demonstrates how to use VoxelOps neuroimaging pipelines (HeudiConv, QSIPrep, QSIRecon) with neuroflow.

## Prerequisites

- VoxelOps installed: `pip install git+https://github.com/yalab-dev/VoxelOps.git`
- neuroflow configured with `neuroflow.yaml`
- DICOM data discovered and validated
- FreeSurfer license for QSI pipelines

## Setup and Configuration

In [1]:
# Imports
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML

from neuroflow.adapters.voxelops import VoxelopsAdapter
from neuroflow.config import NeuroflowConfig
from neuroflow.core.state import StateManager
from neuroflow.models.session import Session, SessionStatus
from neuroflow.models.subject import Subject

In [3]:
# Load configuration
CONFIG_PATH = "/home/galkepler/Projects/neuroflow/neuroflow.yaml"  # Update with your config path

config = NeuroflowConfig.from_yaml(CONFIG_PATH)
print(f"✓ Configuration loaded")
print(f"  BIDS root: {config.paths.bids_root}")
print(f"  Derivatives: {config.paths.derivatives}")
print(f"  Database: {config.database.url}")

✓ Configuration loaded
  BIDS root: /media/storage/yalab-dev/snbb_neuroflow/bids
  Derivatives: /media/storage/yalab-dev/snbb_neuroflow/derivatives
  Database: sqlite:////media/storage/yalab-dev/snbb_neuroflow/.neuroflow/neuroflow.db


In [4]:
# Initialize adapter and state manager
adapter = VoxelopsAdapter(config)
state = StateManager(config)

print(f"✓ VoxelOps adapter initialized")
print(f"  VoxelOps available: {adapter._voxelops_available}")

✓ VoxelOps adapter initialized
  VoxelOps available: True


## Explore Database

Let's see what sessions and subjects are in the database.

In [10]:
# Query sessions
with state.get_session() as db:
    sessions = db.query(Session).all()
    
    # Convert to DataFrame for display
    session_data = []
    for s in sessions:
        session_data.append({
            'ID': s.id,
            'Subject': s.subject.participant_id,
            'Session': s.session_id,
            'Status': s.status.value,
            'Valid': s.is_valid,
            'DICOM Path': s.dicom_path,
        })
    
    df_sessions = pd.DataFrame(session_data)

print(f"Found {len(df_sessions)} sessions in database:\n")
display(df_sessions)

Found 2 sessions in database:



,ID,Subject,Session,Status,Valid,DICOM Path
0,1,0250,202007141404,discovered,None,/media/storage/yalab-dev/test_dicom/20200714_1404
1,2,YBH10005,202510030917,discovered,None,/media/storage/yalab-dev/test_dicom/20251003_0917


In [21]:
# Select a session for processing
# Update this with your session ID
SESSION_ID = 1

with state.get_session() as db:
    session = db.get(Session, SESSION_ID)
    if session:
        print(f"Selected Session:")
        print(f"  Subject: {session.subject.participant_id}")
        print(f"  Session: {session.session_id}")
        print(f"  Status: {session.status.value}")
        print(f"  DICOM: {session.dicom_path}")
        SUBJECT_ID = session.subject_id
    else:
        print(f"ERROR: Session {SESSION_ID} not found")

Selected Session:
  Subject: 0250
  Session: 202007141404
  Status: discovered
  DICOM: /media/storage/yalab-dev/test_dicom/20200714_1404


## Pipeline 1: BIDS Conversion with HeudiConv

Convert DICOM files to BIDS format.

In [25]:
# Check heuristic configuration
bids_cfg = config.pipelines.bids_conversion
heuristic = bids_cfg.voxelops_config.get('heuristic')

print(f"BIDS Conversion Configuration:")
print(f"  Heuristic: {heuristic}")
print(f"  Validator: {bids_cfg.voxelops_config.get('bids_validator', True)}")
print(f"  Overwrite: {bids_cfg.voxelops_config.get('overwrite', False)}")

# Check heuristic exists
if heuristic and Path(heuristic).exists():
    print(f"  ✓ Heuristic file exists")
else:
    print(f"  ✗ WARNING: Heuristic file not found!")

BIDS Conversion Configuration:
  Heuristic: /etc/neuroflow/heuristics/brainbank.py
  Validator: True
  Overwrite: False
  ✓ Heuristic file exists


In [27]:
# Run BIDS conversion
print(f"Starting BIDS conversion for session {SESSION_ID}...\n")

result = adapter.run("bids_conversion", session_id=SESSION_ID)

# Display results
print("\n" + "=" * 60)
if result.success:
    print("✓ BIDS Conversion Successful!")
    print(f"  Duration: {result.duration_seconds:.1f} seconds")
    print(f"  Output: {result.output_path}")
    
    if result.metrics and result.metrics.get('skipped'):
        print(f"  Note: {result.metrics['reason']}")
else:
    print("✗ BIDS Conversion Failed")
    print(f"  Error: {result.error_message}")
print("=" * 60)

Starting BIDS conversion for session 1...

2026-02-05 22:35:13 [info     ] adapter.run                    pipeline=bids_conversion session_id=1 subject_id=None
2026-02-05 22:35:13 [error    ] adapter.voxelops_failed        error='Instance <Session at 0x751a1bb9dc90> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)' pipeline=bids_conversion


DetachedInstanceError: Instance <Session at 0x751a1bb9dc90> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)

In [ ]:
# Inspect BIDS outputs
if result.success and result.output_path:
    # Find NIfTI files
    nifti_files = list(result.output_path.rglob("*.nii.gz"))
    
    print(f"Found {len(nifti_files)} NIfTI files:\n")
    
    file_data = []
    for f in nifti_files[:10]:  # Show first 10
        rel_path = f.relative_to(result.output_path)
        size_mb = f.stat().st_size / (1024 * 1024)
        file_data.append({
            'File': rel_path.name,
            'Path': str(rel_path.parent),
            'Size (MB)': f"{size_mb:.1f}"
        })
    
    df_files = pd.DataFrame(file_data)
    display(df_files)
    
    if len(nifti_files) > 10:
        print(f"\n... and {len(nifti_files) - 10} more files")

## Pipeline 2: QSIPrep Preprocessing

Preprocess diffusion MRI data.

In [ ]:
# Check QSIPrep configuration
qsiprep_cfg = None
for p in config.pipelines.session_level:
    if p.name == "qsiprep":
        qsiprep_cfg = p
        break

if qsiprep_cfg:
    print(f"QSIPrep Configuration:")
    print(f"  CPUs: {qsiprep_cfg.voxelops_config.get('nprocs', 8)}")
    print(f"  Memory: {qsiprep_cfg.voxelops_config.get('mem_mb', 16000)} MB")
    print(f"  Resolution: {qsiprep_cfg.voxelops_config.get('output_resolution', 1.6)} mm")
    print(f"  FreeSurfer license: {qsiprep_cfg.voxelops_config.get('fs_license')}")
    
    # Check license
    fs_license = qsiprep_cfg.voxelops_config.get('fs_license')
    if fs_license and Path(fs_license).exists():
        print(f"  ✓ FreeSurfer license exists")
    else:
        print(f"  ✗ WARNING: FreeSurfer license not found!")
else:
    print("ERROR: QSIPrep not configured")

In [ ]:
# Run QSIPrep
print(f"Starting QSIPrep preprocessing for session {SESSION_ID}...")
print(f"NOTE: This may take several hours (4-12 hours typical)\n")

# You can override defaults here
result_qsiprep = adapter.run(
    "qsiprep",
    session_id=SESSION_ID,
    # nprocs=16,  # Uncomment to override
    # mem_mb=32000,  # Uncomment to override
)

# Display results
print("\n" + "=" * 60)
if result_qsiprep.success:
    print("✓ QSIPrep Successful!")
    print(f"  Duration: {result_qsiprep.duration_seconds / 60:.1f} minutes")
    print(f"  Output: {result_qsiprep.output_path}")
    
    if result_qsiprep.metrics and result_qsiprep.metrics.get('skipped'):
        print(f"  Note: {result_qsiprep.metrics['reason']}")
else:
    print("✗ QSIPrep Failed")
    print(f"  Error: {result_qsiprep.error_message}")
print("=" * 60)

In [ ]:
# Inspect QSIPrep outputs
if result_qsiprep.success and result_qsiprep.output_path:
    # Find preprocessed DWI files
    preproc_files = list(result_qsiprep.output_path.rglob("*_desc-preproc_dwi.nii.gz"))
    print(f"Preprocessed DWI files: {len(preproc_files)}")
    
    # Find HTML reports
    html_reports = list(result_qsiprep.output_path.rglob("*.html"))
    print(f"Quality reports: {len(html_reports)}")
    
    if html_reports:
        print(f"\nView quality report: {html_reports[0]}")
        print(f"Open in browser: file://{html_reports[0].absolute()}")

## Pipeline 3: QSIRecon Reconstruction

Reconstruct diffusion models and compute connectivity matrices.

In [ ]:
# Check QSIRecon configuration
qsirecon_cfg = None
for p in config.pipelines.subject_level:
    if p.name == "qsirecon":
        qsirecon_cfg = p
        break

if qsirecon_cfg:
    print(f"QSIRecon Configuration:")
    print(f"  CPUs: {qsirecon_cfg.voxelops_config.get('nprocs', 8)}")
    print(f"  Memory: {qsirecon_cfg.voxelops_config.get('mem_mb', 16000)} MB")
    print(f"  Recon spec: {qsirecon_cfg.voxelops_config.get('recon_spec', 'default')}")
    print(f"  Atlases: {qsirecon_cfg.voxelops_config.get('atlases', [])}")
else:
    print("ERROR: QSIRecon not configured")

In [ ]:
# Run QSIRecon
print(f"Starting QSIRecon reconstruction for subject {SUBJECT_ID}...")
print(f"NOTE: This may take several hours (2-8 hours typical)\n")

result_qsirecon = adapter.run(
    "qsirecon",
    subject_id=SUBJECT_ID,
)

# Display results
print("\n" + "=" * 60)
if result_qsirecon.success:
    print("✓ QSIRecon Successful!")
    print(f"  Duration: {result_qsirecon.duration_seconds / 60:.1f} minutes")
    print(f"  Output: {result_qsirecon.output_path}")
    
    if result_qsirecon.metrics and result_qsirecon.metrics.get('skipped'):
        print(f"  Note: {result_qsirecon.metrics['reason']}")
else:
    print("✗ QSIRecon Failed")
    print(f"  Error: {result_qsirecon.error_message}")
print("=" * 60)

In [ ]:
# Inspect QSIRecon outputs
if result_qsirecon.success and result_qsirecon.output_path:
    # Find connectivity matrices
    conn_matrices = list(result_qsirecon.output_path.rglob("*_conndata-*.csv"))
    print(f"Connectivity matrices: {len(conn_matrices)}")
    
    if conn_matrices:
        print(f"\nConnectivity matrices found:")
        for cm in conn_matrices[:5]:
            print(f"  - {cm.name}")
        
        # Load and display one
        if len(conn_matrices) > 0:
            print(f"\nLoading: {conn_matrices[0].name}")
            conn_df = pd.read_csv(conn_matrices[0])
            print(f"  Shape: {conn_df.shape}")
            print(f"  Regions: {conn_df.shape[0]}")
            display(conn_df.head())

## Visualization: Connectivity Matrix

Visualize a connectivity matrix.

In [ ]:
# Optional: Visualize connectivity matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

if result_qsirecon.success and result_qsirecon.output_path:
    conn_matrices = list(result_qsirecon.output_path.rglob("*_conndata-*.csv"))
    
    if conn_matrices:
        # Load connectivity matrix
        conn_df = pd.read_csv(conn_matrices[0], index_col=0)
        conn_matrix = conn_df.values
        
        # Plot
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Log scale for better visualization
        conn_matrix_log = np.log10(conn_matrix + 1)
        
        im = ax.imshow(conn_matrix_log, cmap='hot', aspect='auto')
        ax.set_title(f'Connectivity Matrix - {conn_matrices[0].name}', fontsize=14)
        ax.set_xlabel('Region', fontsize=12)
        ax.set_ylabel('Region', fontsize=12)
        
        # Colorbar
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label('log10(connectivity + 1)', fontsize=12)
        
        plt.tight_layout()
        plt.show()
        
        # Basic statistics
        print(f"\nConnectivity Statistics:")
        print(f"  Mean: {conn_matrix.mean():.4f}")
        print(f"  Std: {conn_matrix.std():.4f}")
        print(f"  Min: {conn_matrix.min():.4f}")
        print(f"  Max: {conn_matrix.max():.4f}")

## Summary

Review all pipeline results.

In [ ]:
# Compile results
summary_data = []

if 'result' in locals():
    summary_data.append({
        'Pipeline': 'BIDS Conversion',
        'Success': '✓' if result.success else '✗',
        'Duration (min)': f"{result.duration_seconds / 60:.1f}" if result.duration_seconds else 'N/A',
        'Output': str(result.output_path) if result.output_path else 'N/A'
    })

if 'result_qsiprep' in locals():
    summary_data.append({
        'Pipeline': 'QSIPrep',
        'Success': '✓' if result_qsiprep.success else '✗',
        'Duration (min)': f"{result_qsiprep.duration_seconds / 60:.1f}" if result_qsiprep.duration_seconds else 'N/A',
        'Output': str(result_qsiprep.output_path) if result_qsiprep.output_path else 'N/A'
    })

if 'result_qsirecon' in locals():
    summary_data.append({
        'Pipeline': 'QSIRecon',
        'Success': '✓' if result_qsirecon.success else '✗',
        'Duration (min)': f"{result_qsirecon.duration_seconds / 60:.1f}" if result_qsirecon.duration_seconds else 'N/A',
        'Output': str(result_qsirecon.output_path) if result_qsirecon.output_path else 'N/A'
    })

df_summary = pd.DataFrame(summary_data)

print("\n" + "=" * 70)
print("PIPELINE EXECUTION SUMMARY")
print("=" * 70)
display(df_summary)
print("=" * 70)

## Next Steps

1. **Quality Check**: Review HTML reports from QSIPrep
2. **Group Analysis**: Run analysis across multiple subjects
3. **Connectivity Analysis**: Analyze connectivity matrices
4. **Visualization**: Create publication-ready figures

## Additional Resources

- [VoxelOps Usage Guide](../../docs/voxelops-usage.md)
- [Example Scripts](./README.md)
- [QSIPrep Documentation](https://qsiprep.readthedocs.io/)
- [BIDS Specification](https://bids-specification.readthedocs.io/)